# Normalization stats per timestamp 
- Here we calculate some raster's stats. This is really useful to check that all patchlets have valid data. In the future, this stats will be use to normalize the used data.


In [ ]:
import os
import shutil

print(os.getcwd())

In [ ]:
from functools import partial
from concurrent.futures import ProcessPoolExecutor

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from tqdm.auto import tqdm

from fd.compute_normalization import (ComputeNormalizationConfig, 
                                      stats_per_npz_ts, 
                                      prepare_filesystem,
                                      concat_npz_results,
                                      create_per_band_norm_dataframe)
from fd.utils import multiprocess

## Config

In [ ]:
save_patchlet_npz = '/data/lscalambrin/proyecto_integrador/segmentation/pergamino/patchlets_npz'
df_path = '/data/lscalambrin/proyecto_integrador/segmentation/pergamino/patchlet-info.csv'

config = ComputeNormalizationConfig(
    bucket_name='bucket-name',
    aws_access_key_id='',
    aws_secret_access_key='',
    aws_region='eu-central-1',
    npz_files_folder=save_patchlet_npz,
    metadata_file=df_path)

In [ ]:
npz_files = os.listdir(config.npz_files_folder)

In [ ]:
partial_fn = partial(stats_per_npz_ts, config=config)
results = multiprocess(partial_fn, npz_files, max_workers=24)

In [ ]:
# choose here which stats you are interested in from
stats_keys = ['mean', 'std', 'median', 'perc_99']
identifier_keys = ['timestamp', 'patchlet'] 

concatenated_stats = {}

for key in stats_keys+identifier_keys: 
    concatenated_stats[key] = concat_npz_results(key, results)

In [ ]:
df = create_per_band_norm_dataframe(concatenated_stats, stats_keys, identifier_keys)

In [ ]:
df[df.mean_b0==0]

In [ ]:
### Block to delete bad data. If the above dataframe is not empty, this code should be executed and also notebook 05 and 06(again),
### so previously we need to remove npz directory, which is "/data/lscalambrin/proyecto_integrador/segmentation/pergamino/patchlets_npz/" 
### for pergamino region
# tmp = df[df.mean_b0==0]
# tmp_list = tmp.patchlet.tolist()
# mylist = list( dict.fromkeys(tmp_list) )
# for dir_patchlet in mylist:
#     if os.path.exists(dir_patchlet):
#         shutil.rmtree(dir_patchlet)
#         print('in')

## Analysis

In [ ]:
# convert to datetime
timestamps = df['timestamp'].apply(lambda d: d.tz_localize(None))
df['timestamp']=timestamps.astype(np.datetime64)

# add "month" period
df['month']=df.timestamp.dt.to_period("M")

In [ ]:
def plot_distributions(dataframe, stat, stat_title=None):
    colors = ['b','g','r','y']
    bands = list(range(4))
    
    if not stat_title:
        stat_title = stat

    log=True
    fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(18,13))
    for band in bands:
        dataframe.hist(f'{stat}_b{band}', ax=ax[0], range=(0,10000),
                       bins=100, log=log, color=colors[band], 
                       alpha=0.3, label=f'b{band}')
    ax[0].legend()
    ax[0].grid(axis='x')
    ax[0].set_title(f'Histograms of {stat_title}');

    log=False
    for band in bands:
        dataframe.hist(f'{stat}_b{band}', cumulative=True,  range=(0,10000),
                       density=True, ax=ax[1], bins=100, log=log, 
                       color=colors[band], alpha=0.3, label=f'b{band}')
    ax[1].legend()
    ax[1].grid(axis='x')
    ax[1].set_title(f'Cumulative distributions of {stat_title}');

In [ ]:
plot_distributions(df, 'mean','means')

### How do distributions change through time?

In [ ]:
aggs = {}
stat_cols = []
stats = ['perc_99', 'mean', 'median', 'std']
bands = list(range(4))
for stat in stats:
    for band in bands:
        aggs[f'{stat}_b{band}'] = [np.std, np.mean];
        stat_cols.append(f'{stat}_b{band}')

In [ ]:
monthly = pd.DataFrame(df.groupby('month', as_index=False)[stat_cols].agg(aggs))
monthly.columns = ['_'.join(col).strip() for col in monthly.columns.values]
monthly.rename(columns={'month_':'month'}, inplace=True)

In [ ]:
monthly

In [1]:
def monthly_stats(monthly_df, stat, stat_title=None):
    fig, ax = plt.subplots(figsize=(12,9))
    cols = ['b','g','r','y']
    bands = ['Azul', 'Verde', 'Roja', 'NIR']
    if not stat_title:
        stat_title = stat
        
    for band in range(4):
        x_vals = np.array([m.month if m.month>=9 else m.month+12 for m in monthly_df['month']])

        ax.plot(x_vals, monthly_df[f'{stat}_b{band}_mean'].values, 
                color=cols[band], label=f'Banda {bands[band]}')
        
        ax.scatter(x_vals, monthly_df[f'{stat}_b{band}_mean'].values, 
                color=cols[band])
        ax.fill_between(x_vals, 
                        monthly_df[f'{stat}_b{band}_mean'].values - 
                        monthly_df[f'{stat}_b{band}_std'].values, 
                        monthly_df[f'{stat}_b{band}_mean'].values + 
                        monthly_df[f'{stat}_b{band}_std'].values, color=cols[band], 
                        alpha=0.2)
        str_month_list = ['a','Sep','Oct','Nov','Dic','En', 'Feb','Mar']
#         ax.set_xticks(range(9,15) 
        ax.set_xticklabels(str_month_list)
    ax.tick_params(direction='out', length=6, width=2, grid_alpha=0.5,labelsize = 15)
    ax.legend(fontsize = 15)
    ax.grid()
    ax.set_title(f'{stat_title} through months')
#     ax.set_title('Valor medio', fontsize = 19)
    ax.set_title('Desviación estándar', fontsize = 19)

In [ ]:
monthly_stats(monthly, 'mean', 'means')

In [ ]:
monthly_stats(monthly, 'std', 'standard deviations')

## Normalization factors per month per band

We calculate normalization factors for two different normalizations

In [ ]:
norm_cols = [norm.format(band) 
             for norm in ['perc_99_b{0}_mean', 
                          'mean_b{0}_mean', 
                          'median_b{0}_mean', 
                          'std_b{0}_mean'] for band in range(4)]

def norms(month):
    return monthly.loc[monthly.month==month][norm_cols].values[0]

In [ ]:
df['norm_perc99_b0'], df['norm_perc99_b1'], df['norm_perc99_b2'], df['norm_perc99_b3'], \
df['norm_meanstd_mean_b0'], df['norm_meanstd_mean_b1'], df['norm_meanstd_mean_b2'], df['norm_meanstd_mean_b3'], \
df['norm_meanstd_median_b0'], df['norm_meanstd_median_b1'], df['norm_meanstd_median_b2'], df['norm_meanstd_median_b3'], \
df['norm_meanstd_std_b0'], df['norm_meanstd_std_b1'], df['norm_meanstd_std_b2'], df['norm_meanstd_std_b3'] = zip(*map(norms, df.month))

In [ ]:
# another check; should be similar to `monthly_stats(monthly, 'mean','means')`
df[['month','norm_meanstd_mean_b0','norm_meanstd_mean_b1','norm_meanstd_mean_b2','norm_meanstd_mean_b3']].drop_duplicates().reset_index(drop=True).plot()

## Add this info to patchlet info

In [ ]:
df.columns

In [ ]:
len(df)

In [ ]:
with open(config.metadata_file, 'rb') as fcsv:
    df_info = pd.read_csv(fcsv)

In [ ]:
df_info['timestamp'] = pd.to_datetime(df_info.timestamp)

In [ ]:
timestamps = df_info['timestamp'].apply(lambda d: d.tz_localize(None))
df_info['timestamp'] = timestamps.astype(np.datetime64)

In [ ]:
df_info.head()

In [ ]:
new_df = df_info.merge(df, how='inner', on=['patchlet', 'timestamp'])

In [ ]:
len(new_df)

In [ ]:
new_df.head()

In [ ]:
with open(config.metadata_file, 'w') as fcsv:
    new_df.to_csv(fcsv, index=False)